In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [2]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.0)

In [3]:
llm.invoke("Hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-c4fec6df-223a-49cc-8906-6997962208cb-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [5]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [6]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(in

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
search_tool = TavilySearchResults()

In [9]:
search_tool.invoke("Recipes of pancakes", max_results=3)

[{'url': 'https://www.thekitchenmagpie.com/pancakes-from-scratch/',
  'content': 'In a medium bowl or a large 16 cup capacity liquid measuring cup, whisk together the flour, baking powder, white sugar and salt. In another large capacity liquid measuring cup, combine 2 1/2 cups of milk, eggs, oil and vanilla. Mix until combined. If you need to, add more milk until the batter reaches desired consistency.'},
 {'url': 'https://www.allrecipes.com/recipe/21014/good-old-fashioned-pancakes/',
  'content': '"\nEditorial contributions by Corey Williams\nIngredients\n1 ½ cups all-purpose flour\n3 ½ teaspoons baking powder\n1 tablespoon white sugar\n¼ teaspoon salt, or more to taste\n1 ¼ cups milk\n3 tablespoons butter, melted\n1\negg\nDirections\nSift flour, baking powder, sugar, and salt together in a large bowl. How to Make Pancakes\nYou\'ll find a detailed ingredient list and step-by-step instructions in the recipe below, but let\'s go over the basics:\nPancake Ingredients\nYou likely already 

1. [See here for the quick setup for LANGSMITH](https://docs.smith.langchain.com/).
2. [See here for the quick setup for TAVILY API KEY](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/).

In [10]:
tools = [search_tool]

In [11]:
from langchain.agents import create_tool_calling_agent

In [14]:
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789

In [17]:
agent.invoke({"input": "Search the web for recipes for pancakes", "intermediate_steps": []})

[ToolAgentAction(tool='tavily_search_results_json', tool_input={'query': 'pancake recipes'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'pancake recipes'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_D07mZnLKeX5WOfaIBcRs3IWZ', 'function': {'arguments': '{"query":"pancake recipes"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 92, 'total_tokens': 113}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1e6cc8d5-1639-4825-9546-36b971ec83b6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'pancake recipes'}, 'id': 'call_D07mZnLKeX5WOfaIBcRs3IWZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 21, 'total_tokens': 113})], tool_call_id='call_D07mZnLKeX5WOfaIBcRs3IWZ')]

In [18]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_

In [21]:
agent_executor.invoke({"input": "Search the web for recipes for pancakes"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'pancake recipes'}`


[{'url': 'https://www.allrecipes.com/recipe/21014/good-old-fashioned-pancakes/', 'content': '"\nEditorial contributions by Corey Williams\nIngredients\n1 ½ cups all-purpose flour\n3 ½ teaspoons baking powder\n1 tablespoon white sugar\n¼ teaspoon salt, or more to taste\n1 ¼ cups milk\n3 tablespoons butter, melted\n1\negg\nDirections\nSift flour, baking powder, sugar, and salt together in a large bowl. How to Make Pancakes\nYou\'ll find a detailed ingredient list and step-by-step instructions in the recipe below, but let\'s go over the basics:\nPancake Ingredients\nYou likely already have everything you need to make this pancake recipe. Yes, you can freeze pancakes and pancake batter.\n· To freeze cooked pancakes: Allow the pancakes to cool completely, then arrange them in a single layer on a baking sheet (make sure the edges aren\'t touching). All Rights Reserved\nPhotos

{'input': 'Search the web for recipes for pancakes',
 'output': 'Here are some great pancake recipes you can try:\n\n1. **Good Old-Fashioned Pancakes**  \n   - **Ingredients**: 1 ½ cups all-purpose flour, 3 ½ teaspoons baking powder, 1 tablespoon white sugar, ¼ teaspoon salt, 1 ¼ cups milk, 3 tablespoons butter (melted), 1 egg.  \n   - **Instructions**: Sift flour, baking powder, sugar, and salt together in a large bowl. Mix in the milk, melted butter, and egg until smooth. Cook on a griddle until golden brown.  \n   - [View Recipe](https://www.allrecipes.com/recipe/21014/good-old-fashioned-pancakes/)\n\n2. **Light and Fluffy Pancakes from Scratch**  \n   - This recipe focuses on making light and fluffy pancakes with just a few ingredients.  \n   - [View Recipe](https://www.thekitchenmagpie.com/pancakes-from-scratch/)\n\n3. **Best Fluffy Pancakes**  \n   - This recipe includes variations like 3-Ingredient Pancakes and Fluffy Blueberry Pancakes.  \n   - [View Recipe](https://cafedelites

# TM's input: 'Cooking pancakes in a low gravity situation, like on the moon, or even zero gravity on ISS'

In [22]:
query = 'Cooking pancakes in a low gravity situation, like on the moon, or even zero gravity on ISS'

output = agent_executor.invoke({"input": query})

output



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'cooking pancakes in low gravity moon ISS'}`


[{'url': 'https://www.sciencedirect.com/science/article/pii/S2214799323000358', 'content': 'The main aspect of these conditions is reduced gravity. In the present work, at first, the ways of achieving low-gravity conditions on earth are presented. It appears that there is little possibility to use these ways effectively to perform long-enough cooking tests in low gravity, with ISS being currently the only reliable alternative.'}, {'url': 'https://www.esa.int/Science_Exploration/Human_and_Robotic_Exploration/Research/Flying_frying_in_microgravity', 'content': 'ESA has been supporting research in frying cooking methods in microgravity to fill in knowledge gaps on Earth and in space. Even though frying potatoes is done everywhere around the world, it involves complex physics and chemistry, and in space everything becomes more complicated. It was no

{'input': 'Cooking pancakes in a low gravity situation, like on the moon, or even zero gravity on ISS',
 'output': "Cooking pancakes in low or zero gravity, such as on the Moon or the International Space Station (ISS), presents unique challenges and considerations due to the absence of gravitational forces. Here are some insights based on current research and practices:\n\n1. **Cooking Methods**: Traditional cooking methods that rely on gravity, such as flipping pancakes, are not feasible in microgravity. Instead, astronauts may need to use specialized equipment that can contain the batter and apply heat without the need for flipping. For example, frying methods have been researched to understand how they can be adapted for space cooking.\n\n2. **Research and Development**: The European Space Agency (ESA) has been conducting research on frying and cooking methods in microgravity. This research aims to fill knowledge gaps about how cooking processes change in space, as the physics and c

In [23]:
query_specific_website = "Research only medium.com domains for articles about using LLM agents to augment human research workflows."

output = agent_executor.invoke({"input": query_specific_website})
output



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'site:medium.com using LLM agents to augment human research workflows'}`


[{'url': 'https://vijaykumarkartha.medium.com/multiple-ai-agents-creating-multi-agent-workflows-using-langgraph-and-langchain-0587406ec4e6', 'content': "Collaborative Multi-Agents. Much like human collaboration, different AI agents in a collaborative multi-agent workflow communicate using a shared scratchpad of messages. This allows each agent to view other agents' work and observe all the individual steps taken. The various AI agents could be based on the same LLM but in different roles."}, {'url': 'https://medium.com/@kose2livs/enhancing-llm-performance-with-agents-workflow-1c7db07d7947', 'content': 'In such a workflow, we can ask the LLM to iterate on documents multiple times to produce high-quality answers. ... while using the Agent workflow, some agents even reached 95% accuracy on GPT-3.5 ...'}, {'url': 'https:/

{'input': 'Research only medium.com domains for articles about using LLM agents to augment human research workflows.',
 'output': "Here are some articles from Medium.com that discuss using LLM (Large Language Model) agents to augment human research workflows:\n\n1. **[Multiple AI Agents: Creating Multi-Agent Workflows Using LangGraph and LangChain](https://vijaykumarkartha.medium.com/multiple-ai-agents-creating-multi-agent-workflows-using-langgraph-and-langchain-0587406ec4e6)**  \n   This article explores collaborative multi-agent workflows where different AI agents communicate using a shared scratchpad of messages, allowing them to observe each other's work and steps taken.\n\n2. **[Enhancing LLM Performance with Agents Workflow](https://medium.com/@kose2livs/enhancing-llm-performance-with-agents-workflow-1c7db07d7947)**  \n   It discusses how LLMs can iterate on documents multiple times to produce high-quality answers, with some agents achieving up to 95% accuracy using the Agent wor